In [49]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

import torch.utils.data as Data

## 3.3.1 生成数据集

In [50]:
num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2


# 数据生成
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)),
                        dtype=torch.float
                       )


labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), 
                       dtype=torch.float
                      )

## 3.3.2 读取数据

In [48]:
batch_size = 10

# 读取数据
dataSet = Data.TensorDataset(features, labels)

# 随机读取小批量
data_iter = Data.DataLoader(dataSet, batch_size, shuffle=True)

In [54]:
for X, y in data_iter:
    print("X: ", X, '\n y: ', y)
    break

X:  tensor([[ 0.3421, -0.2737],
        [-0.4767,  0.3001],
        [-0.6906, -1.1004],
        [ 2.0606,  0.8971],
        [ 0.1806,  1.1221],
        [ 0.3102, -0.2225],
        [-0.1582, -0.9630],
        [ 0.9650,  0.2398],
        [ 0.8849,  0.3453],
        [-1.7051,  1.7886]]) 
 y:  tensor([ 5.8125,  2.2182,  6.5398,  5.2610,  0.7453,  5.5861,  7.1576,  5.2930,
         4.7983, -5.2805])


## 3.3.3 定义模型

In [55]:
from torch import nn

class LinearNet(nn.Module):
    
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
        
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [56]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

In [57]:
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [58]:
net[0]

Linear(in_features=2, out_features=1, bias=True)

In [59]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0403, -0.3327]], requires_grad=True)
Parameter containing:
tensor([-0.5807], requires_grad=True)


## 3.3.4 初始化模型参数

In [64]:
from torch.nn import init


init.normal_(net[0].weight, mean=0, std=0.01)

# 也可以直接修改bias的data: net[0].bias.data.fill_(0)
init.constant_(net[0].bias, val=0)  

Parameter containing:
tensor([0.], requires_grad=True)

In [65]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0258, 0.0094]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [66]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [70]:
import torch.optim as optim


optimizer = optim.SGD(net.parameters(), lr=0.03)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

## 3.3.7 训练模型

In [44]:
num_epochs = 3


for epoch in range(1,num_epochs + 1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output,y.view(-1,1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        pass
    
    print(f'epoch {epoch}, loss {l.item()}')

epoch 1, loss 7.136028580134735e-05
epoch 2, loss 0.00010197574738413095
epoch 3, loss 3.388420009287074e-05


In [45]:
dense = net[0]

print('true_w: ', true_w, dense.weight)
print("")
print('true_b: ', true_b, dense.bias)

true_w:  [2, -3.4] Parameter containing:
tensor([[ 2.0000, -3.4003]], requires_grad=True)

true_b:  4.2 Parameter containing:
tensor([4.2001], requires_grad=True)
